In [1]:
def covariate_type(i, n, K, num):
    '''
    Returns the covariate type of unit i. Assumes that the number of communities is divisible by the number of covariate types.

    n (int) = number of units in the population
    K (int) = number of communities (blocks) in the SBM
    num (int)  = number of covariate types

    Example: Suppose n=8, K=4, and num = 2. 
        Communities 0 and 1 are assigned covariate type 0 and contain individuals 0,1,2,3
        Communities 2 and 3 are assigned covariate type 1 and contain individuals 4,5,6,7
        Individual i's covariate type is i // 4. Note that 4 = n // num.

    Example: Suppose n=8, K=2, and num = 2.
        Community 0 is assigned covariate type 0 and contains individuals 0,1,2,3
        Communities 1 is assigned covariate type 1 and contains individuals 4,5,6,7
        Individual i's covariate type is i // 4. Note that 4 = n // num.

    Example: Suppose n=8, K=4, and num = 4.
        Community 0 is assigned covariate type 0 and contains individuals 0,1
        Community 1 is assigned covariate type 1 and contains individuals 2,3
        Community 2 is assigned covariate type 2 and contains individuals 4,5
        Community 3 is assigned covariate type 3 and contains individuals 6,7
        Individual i's covariate type is i // 2. Note that 2 = n // num.
    '''
    assert num <= K and num%K==0, "there cannot be more covariate types than number of communities; number of types should divide evenly into the number of groups"
    div = n // num
    return i // div

In [2]:
n = 25
K = 5
num = 5

for i in range(n):
    print(covariate_type(i, n, K, num))

0
0
0
0
0
1
1
1
1
1
2
2
2
2
2
3
3
3
3
3
4
4
4
4
4


In [2]:
import numpy as np
C = np.zeros((5,5))
print(C)

[[0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]]


In [12]:
arr = np.zeros(10)
arr[0:10//2] = 1
print(arr)
arr[10//2:] = 2
print(arr)

In [18]:
x1 = np.arange(9.0).reshape((3, 3))
x2 = np.arange(3.0)
print(x1)
print()
print(x2)
print()
print(np.multiply(x1, x2))

[[0. 1. 2.]
 [3. 4. 5.]
 [6. 7. 8.]]

[0. 1. 2.]

[[ 0.  1.  4.]
 [ 0.  4. 10.]
 [ 0.  7. 16.]]


In [3]:
arr = np.array([0,0,0,0,1,1,1,1])
P = np.random.rand(8)
print(arr)
print(P)

[0 0 0 0 1 1 1 1]
[0.10496053 0.81057902 0.03369502 0.0766198  0.29896807 0.22788365
 0.3027362  0.71794009]


In [4]:
P = (P < 0.5)+0
print(P)
np.nonzero(P)

[1 0 1 1 1 1 1 0]


(array([0, 2, 3, 4, 5, 6]),)

In [9]:
arr = np.zeros(8) + 0.2
print(arr)

arr[np.nonzero(P)] = 0.8
print(arr)

[0.2 0.2 0.2 0.2 0.2 0.2 0.2 0.2]
[0.8 0.2 0.8 0.8 0.8 0.8 0.8 0.2]


In [13]:
arr2 = np.zeros(8) + 0.8
arr3 = np.concatenate((arr,arr2))
print(arr3)


[0.8 0.2 0.8 0.8 0.8 0.8 0.8 0.2 0.8 0.8 0.8 0.8 0.8 0.8 0.8 0.8]


In [15]:
np.round(0, 2)

0

In [15]:
import numpy as np
x1 = np.random.rand(1000,1000)
print(x1.shape)
x2 = np.random.rand(1000,)
print(x2.shape)
x1x2 = np.multiply(x1, x2)
print(x1x2.shape)

(1000, 1000)
(1000,)
(1000, 1000)


In [29]:
x1x2dot = np.dot(x1,x2)
x1x2dot.shape

(1000,)

In [11]:
import networkx as nx
def SBM(n, k, Pii, Pij):
    '''
    Returns the adjacency matrix of a stochastic block model on n nodes with k communities
    The edge prob within the same community is Pii
    The edge prob across different communities is Pij
    '''
    sizes = np.zeros(k, dtype=int) + n//k
    probs = np.zeros((k,k)) + Pij
    np.fill_diagonal(probs, Pii)
    G = nx.stochastic_block_model(sizes, probs)
    A = nx.adjacency_matrix(nx.stochastic_block_model(sizes, probs))
    #blocks = nx.get_node_attributes(G, "block")
    return G, A

In [22]:
from myFunctions import simpleWeights
n, nc = 10, 5
Pii, Pij = 0.5,0
diag = 1
offdiag = 1.25
G, A = SBM(n, nc, Pii, Pij)
rand_wts = np.random.rand(n,3)
C = simpleWeights(A, diag, offdiag, rand_wts[:,1].flatten(), rand_wts[:,2].flatten())

In [27]:
print(C.shape)

(10, 10)


In [25]:
minimal = 1/4
extreme = 4
phi = 0.25

n = C.shape[0]
scaling1 = np.zeros(n//2) + minimal
scaling2 = np.zeros(n//2) + extreme

R1 = np.random.rand(n//2)
R2 = np.random.rand(n//2)
R1 = (R1 < phi) + 0
R2 = (R2 < phi) + 0

scaling1[np.nonzero(R1)] = extreme
scaling2[np.nonzero(R2)] = minimal

scaling = np.concatenate((scaling1,scaling2))
print(scaling, scaling.shape)

[0.25 0.25 0.25 0.25 4.   4.   4.   4.   4.   4.  ] (10,)


In [30]:
newC = C.multiply(scaling)
print(newC.shape)

(10, 10)


In [34]:
arr = np.array([[1,2,3], [0,1,0], [1,0,0], [0,0,3]])
print(arr, arr.shape)

[[1 2 3]
 [0 1 0]
 [1 0 0]
 [0 0 3]] (4, 3)


In [36]:
from scipy import sparse
sA = sparse.csr_matrix(arr)
print(sA, sA.shape)

  (0, 0)	1
  (0, 1)	2
  (0, 2)	3
  (1, 1)	1
  (2, 0)	1
  (3, 2)	3 (4, 3)


In [37]:
B = np.array([2, 3, 1])
arrB = np.multiply(arr,B)
print(arrB, arrB.shape)

[[2 6 3]
 [0 3 0]
 [2 0 0]
 [0 0 3]] (4, 3)


In [38]:
AB = sA.multiply(B)
print(AB, AB.shape)

  (0, 0)	2
  (0, 1)	6
  (0, 2)	3
  (1, 1)	3
  (2, 0)	2
  (3, 2)	3 (4, 3)
